In [61]:
from pathlib import Path
import numpy as np
import itertools

In [441]:
def read(prefix='data'):
    data = Path(f'{prefix}/17.txt').read_text().rstrip()
    return data

In [442]:
disp = lambda B, h=10 : print(*[''.join(r) for r in B[-h:]], sep='\n')

In [443]:
def board(count=2022):
    B = np.full((count*4+10,9), fill_value='.')
    B[:,0] = '|'
    B[:,-1] = '|'
    B[-1,:] = '-'
    B[-1,0] = '+'
    B[-1,-1] = '+'
    return B

In [444]:
def shapes(count=2022):
    S = {}
    S['-'] = np.array([['#','#','#','#']])
    S['+'] = np.array([['.','#','.'],['#', '#', '#'], ['.','#','.']])
    S['L'] = np.array([['.','.','#'],['.','.','#'],['#', '#', '#']])
    S['|'] = np.array([['#','#','#','#']]).T
    S['o'] = np.array([['#', '#'], ['#', '#']])
    return S

In [445]:
for s in shapes().values():
    print()
    disp(s)


####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##


In [446]:
disp(board(), h=10)

|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
+-------+


In [447]:
def outofbounds(B, S, x, y):
    if x < 1:
        return True
    if y < 0:
        return True
    if x + S.shape[1] >= B.shape[1]:
        return True
    if y + S.shape[0] >= B.shape[0]:
        return True

    return False

In [448]:
def intersect(B, s, x, y):
    Bw = B[y:y+s.shape[0],x:x+s.shape[1]]
    return ((Bw == '#') & (s == '#')).any()

In [449]:
def place(B, s, x, y):
    Bw = B[y:y+s.shape[0],x:x+s.shape[1]]
    B[y:y+s.shape[0],x:x+s.shape[1]] = np.where((s == '#'), s, Bw)

In [450]:
def drop(B, s, M, M_idx, h):
    x = 3
    y = h - s.shape[0] - 3
    down = False
    while True:
        if down:
            xn, yn = x, y+1
        if not down:
            xn = x+1 if (M[M_idx] == '>') else x-1
            yn = y
            M_idx = (M_idx + 1) % len(M)
        oob = outofbounds(B,s, xn,yn)
        isct = intersect(B,s, xn, yn)
        if oob or isct:
            if down:
                break
        else:
            x, y = xn, yn

        down = not down

    place(B, s, x, y)
    h = min(h, y)
    return h, M_idx


In [464]:
from collections import defaultdict

def compute(M,count, M_idx=0):
    B = board(count)
    S = shapes()
    h = B.shape[0]-1 # floor
    seen = defaultdict(list)
    for i, (k,s) in zip(range(count), itertools.cycle(S.items())):
        h, M_idx = drop(B, s, M, M_idx, h)
        mins = tuple((B[:,1:-1] == '#').argmax(axis=0) - h)
        seen[mins, k, M_idx].append((B.shape[0] - h -1, i))
    return h,B, seen

In [465]:
count = 2022
M = read('test')
h, B, seen = compute(M,count)
print(B.shape[0] - h -1)
disp(B,h=20)
reps = {k: v for (k,v) in seen.items() if len(v) > 1}
len(reps)

3068
|...#...|
|#..####|
|#...#..|
|#...#..|
|#...##.|
|##..##.|
|######.|
|.###...|
|..#....|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+


35

In [468]:
print(*reps.keys(), sep='\n')

((2, 2, 0, 2, 3, 5, 7), 'L', 28)
((0, 4, 2, 4, 5, 7, 9), '|', 34)
((0, 4, 2, 4, 5, 5, 5), 'o', 5)
((0, 1, 1, 1, 1, 5, 5), '-', 11)
((2, 3, 3, 1, 0, 1, 7), '+', 16)
((5, 6, 2, 2, 0, 4, 10), 'L', 20)
((9, 10, 6, 6, 0, 8, 14), '|', 24)
((9, 4, 4, 6, 0, 8, 14), 'o', 34)
((10, 0, 0, 0, 0, 9, 15), '-', 38)
((12, 2, 2, 2, 1, 0, 1), '+', 3)
((13, 2, 2, 0, 2, 1, 2), 'L', 9)
((15, 0, 4, 2, 4, 3, 4), '|', 15)
((15, 0, 0, 0, 4, 3, 4), 'o', 21)
((16, 1, 0, 0, 0, 0, 5), '-', 25)
((1, 0, 1, 2, 2, 2, 7), '+', 30)
((2, 1, 2, 3, 2, 2, 0), 'L', 36)
((4, 3, 4, 5, 0, 4, 2), '|', 2)
((4, 3, 2, 2, 0, 4, 2), 'o', 10)
((5, 4, 0, 0, 0, 0, 3), '-', 14)
((8, 7, 3, 3, 1, 0, 1), '+', 18)
((10, 9, 2, 2, 0, 2, 3), 'L', 23)
((10, 5, 2, 2, 0, 2, 3), '|', 36)
((10, 5, 0, 0, 0, 2, 3), 'o', 2)
((11, 6, 0, 0, 0, 0, 4), '-', 6)
((14, 9, 1, 0, 1, 3, 7), '+', 10)
((17, 12, 2, 2, 0, 6, 10), 'L', 14)
((21, 16, 6, 6, 0, 10, 14), '|', 18)
((21, 4, 4, 6, 0, 10, 14), 'o', 28)
((22, 0, 0, 0, 0, 11, 15), '-', 32)
((24, 2, 2, 2, 1, 0,

In [489]:
count = 5000
M = read()
h, B, seen = compute(M,count)
print(B.shape[0] - h -1)
reps = {k: v for (k,v) in seen.items() if len(v) > 2}
len(reps)

7633


973

In [490]:
reps[((2, 1, 2, 3, 2, 2, 0), 'L', 3579)]

[(933, 617), (3551, 2322), (6169, 4027)]

In [496]:
arep = list(reps.items())[1]
arep

(((4, 3, 4, 5, 0, 4, 2), '|', 3585), [(935, 618), (3553, 2323), (6171, 4028)])

In [500]:
# height, rocks
{(n[0]-c[0],n[1]-c[1]) for rep in reps.items() for (c,n) in zip(rep[1], rep[1][1:])}

{(2618, 1705)}

In [406]:
def cycles(l):
    for ((ch, cm), (nh, nm)) in zip(l,l[1:]):
        return (nh-ch, nm-cm)

In [400]:
def boost(c, target):
    occ0 = c[1][0]
    occ1 = c[1][1]
    cycle_moves = occ1[1]- occ0[1]
    cycle_height = occ1[0] - occ0[0]
    base_height = occ0[0]
    base_moves = occ0[1]
    needed = int((target - base_moves)/ cycle_moves)
    return (base_moves, base_moves + needed*cycle_moves, base_height + needed*cycle_height)

In [506]:
count = 2022
M = read('test')
h, B, seen = compute(M,count)
reps = {k: v for (k,v) in seen.items() if len(v) > 1}
{(n[0]-c[0],n[1]-c[1]) for rep in reps.items() for (c,n) in zip(rep[1], rep[1][1:])}

{(53, 35)}

In [507]:
reps = {k: v for (k,v) in seen.items() if len(v) > 1}
target = 1000000000000
for c in reps.items():
    bm, mv, ht = boost(c, target=target)
    if mv == target:
        print(mv, ht)

1000000000000 1514285714289


In [540]:
count = 10037
M = read()
h, B, seen = compute(M,count)
print(B.shape[0] - h -1)
reps = {k: v for (k,v) in seen.items() if len(v) > 1}
{(n[0]-c[0],n[1]-c[1]) for rep in reps.items() for (c,n) in zip(rep[1], rep[1][1:])}

15375


{(2618, 1705)}

In [541]:
reps = {k: v for (k,v) in seen.items() if len(v) > 1}
target = 10037
for c in reps.items():
    bm, mv, ht = boost(c, target=target)
    if mv == target:
        print(mv, ht)

10037 15378


In [402]:
count = 2022
M = read()
h, B, seen = compute(M,count)
print(B.shape[0] - h -1)
disp(B,h=20)

3085
|###.##.|
|.#..#..|
|#####..|
|..###..|
|...#...|
|..###..|
|...#...|
|..####.|
|..#....|
|..#....|
|#.#....|
|#.#....|
|#.#....|
|#.#....|
|#####..|
|..###..|
|.###...|
|..#....|
|..####.|
+-------+


In [412]:
target = 1000000000000
target/35

28571428571.42857

In [503]:
reps = {k: v for (k,v) in seen.items() if len(v) > 1}
target = 1000000000000
for c in reps.items():
    bm, mv, ht = boost(c, target=target)
    if mv == target:
        print(mv, ht)

1000000000000 1535483870925
